In [1]:
from lxml import etree
import requests
import pandas as pd
import os
from config import *
from bs4 import BeautifulSoup

In [58]:
url = "http://sousuo.gov.cn/data"

HEADER = {
   'Accept-Encoding': 'gzip, deflate',
   'Accept-Language': 'zh-CN,zh;q=0.9',
   'X-Requested-With': 'XMLHttpRequest',
   'Cookie': 'allmobilize=mobile; wdcid=7a24ed37295bf057; wdses=4febaba60cf98871; wdlast=1649517429',
   'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
}

payload = {
    't': 'zhengcelibrary',
    'q': '新能源汽车',
    'timetype': '',
    'mintime': '',
    'maxtime': '',
    'sort': '',
    'sortType': '1',
    'searchfield': '',
    'pcodeJiguan': '',
    'childtype': '',
    'subchildtype': '',
    'tsbq': '',
    'pubtimeyear': '',
    'puborg': '',
    'pcodeYear': '',
    'pcodeNum': '',
    'filetype': '',
    'p': '4',
    'n': '5',
    'inpro': '',
    'bmfl': '',
    'dup': '',
    'orpro': '',
}

topic={
    'gc300':'国务院组织机构',
    'gc188':'综合政务',
    'gc187':'国民经济管理、国有资产监管',
    'gc186':'财政、金融、审计',
    'gc185':'国土资源、能源',
    'gc184':'农业、林业、水利',
    'gc183':'工业、交通',
    'gc182':'商贸、海关、旅游',
    'gc181':'市场监督、安全生产监管',
    'gc180':'城乡建设、环境保护',
    'gc179':'科技、教育',
    'gc178':'文化、广电、新闻出版',
    'gc175':'劳动、人事、监察',
    'gc174':'公安、安全、司法',
    'gc173':'民政、扶贫、救灾',
    'gc172':'民族、宗教',
}
'''
params中的childdtype中，输入topic中的key，可进行主题分类
pubtimeyear：文件发布日期
q: '新能源汽车',即查找与什么相关的内容
'''

"\nparams中的childdtype中，输入topic中的key，可进行主题分类\npubtimeyear：文件发布日期\nq: '新能源汽车',即查找与什么相关的内容\n"

In [59]:
def get_gongwen_content(url,type_of_doc):
    p = requests.get(url, headers = HEADER)
    p = p.text.encode('ISO-8859-1').decode(requests.utils.get_encodings_from_content(p.text)[0])
    tree = etree.HTML(p)
    if type_of_doc == "gongwen":
        document = tree.xpath('//*[@id="UCAP-CONTENT"]//text()')
    else:
        document = tree.xpath('//*[@class="pages_content"]//text()')
        if not len(document)>0:
            document = tree.xpath('//*[@id="UCAP-CONTENT"]//text()')
    doc = ''
    for lists in document:
        doc = doc + lists
    #print(doc)
    return doc

In [60]:
category = []
cat_name = []
content = []
date = []
title = []
summary = []
id_num = []

In [61]:

for year in ['2021','2022']:
    payload['pubtimeyear'] = year
    r = requests.get(url=url,headers=HEADER,params=payload).json()
    total_count = int(r['searchVO']['catMap']['gongwen']['totalCount'])+int(r['searchVO']['catMap']['bumenfile']['totalCount'])
    page = ceil(total_count/5)  
    for i in range(0,page):
        payload['p']=i
        r = requests.get(url=url,headers=HEADER,params=payload).json()
        if r['searchVO']['catMap']['gongwen']['listVO']:
            for web in r['searchVO']['catMap']['gongwen']['listVO']:
                title.append(web['title'])
                date.append(web['pubtimeStr'])
                cat_name.append(web['colname'])
                category.append(web['childtype'])
                summary.append(web['summary'])
                url_of_every_doc = web['url']
                print("gwy:",url_of_every_doc)
                id_num.append(web['id'])
                content.append(get_gongwen_content(url_of_every_doc,"gongwen"))

        if  r['searchVO']['catMap']['bumenfile']['listVO']:
            for web in r['searchVO']['catMap']['bumenfile']['listVO']:
                title.append(web['title'])
                date.append(web['pubtimeStr'])
                cat_name.append(web['colname'])
                category.append(web['childtype'])
                summary.append(web['summary'])
                url_of_every_doc = web['url']
                print("bumen:",url_of_every_doc)
                id_num.append(web['id'])
                content.append(get_gongwen_content(url_of_every_doc,"bumenfile"))

gwy: http://www.gov.cn/zhengce/zhengceku/2021-11/26/content_5653479.htm
gwy: http://www.gov.cn/zhengce/zhengceku/2021-10/26/content_5644984.htm
gwy: http://www.gov.cn/zhengce/zhengceku/2021-05/21/content_5609800.htm
gwy: http://www.gov.cn/zhengce/zhengceku/2021-02/20/content_5587874.htm
gwy: http://www.gov.cn/zhengce/zhengceku/2021-02/22/content_5588274.htm
bumen: http://www.gov.cn/zhengce/zhengceku/2021-12/31/content_5665857.htm
bumen: http://www.gov.cn/zhengce/zhengceku/2021-12/12/content_5660257.htm
bumen: http://www.gov.cn/zhengce/zhengceku/2021-08/26/content_5633554.htm
bumen: http://www.gov.cn/zhengce/zhengceku/2021-05/15/content_5640040.htm
bumen: http://www.gov.cn/zhengce/zhengceku/2021-08/28/content_5633897.htm
gwy: http://www.gov.cn/zhengce/zhengceku/2021-03/25/content_5595644.htm
bumen: http://www.gov.cn/zhengce/zhengceku/2021-03/31/content_5597135.htm
bumen: http://www.gov.cn/zhengce/zhengceku/2021-02/09/content_5586234.htm
bumen: http://www.gov.cn/zhengce/zhengceku/2021-12

In [62]:
df = pd.DataFrame({"category":category, 
"cat_name":cat_name,
"date":date,
"title":title,
"summary":summary,
"id_num":id_num,
"content":content,},
columns =['category','cat_name','date','title','summary','id_num','content'])

In [63]:
SAVE_DIR = "C:/Users/Administrator/Desktop/LDA-crawler/LDA-crawler/data"
os.makedirs(SAVE_DIR, exist_ok=True)
df.to_csv(os.path.join(SAVE_DIR, 'gwy.csv'))